In [5]:
#coding: utf-8

#import des modules
#lecture des json
import json 
#affichage des dictionnaires avec idententation
import pprint
pp = pprint.PrettyPrinter(indent=4, )
#librairie des graphes
import networkx as nx
#affichage des graphs
import matplotlib
import matplotlib.pyplot as plt
#calcul
import numpy as np
from itertools import combinations
#%matplotlib inline

# Toutes les  versions de l'article 9

Nous allons charger les informations contenues dans le fichier article 9:
* la version originale (qui correspond aux statistiques finales de votes sur l'article)
* les versions complémentaires (soit les 108 modifications additionnelles proposées au vote)
selon la même nomenclature

#### Une petite explication sur la génération des slugs
Ils servent de clés au dictionnaire de versions:
Notre but étant de rappatrier les votes de chaque versions  pour chaque participant
il faut identifier de quelle version il s'agit.
Ici pour faciliter le travail et comme on peut voir plus haut
la version originale correspond au nom de générique de l'article

Recapitulons:

L'url de la version originale se présente sous cette forme:
```
urlV_0 = "http://www.republique-numerique.fr/projects/projet-de-loi-numerique/consultation/consultation/opinions/section-2-travaux-de-recherche-et-de-statistique/article-9-acces-aux-travaux-de-la-recherche-financee-par-des-fonds-publics"
```
L'url des versions se présentent toujours sous cette formes
```
urlV_add = "http://www.republique-numerique.fr/projects/projet-de-loi-numerique/consultation/consultation/opinions/section-2-travaux-de-recherche-et-de-statistique/article-9-acces-aux-travaux-de-la-recherche-financee-par-des-fonds-publics/versions/mise-a-disposition-systematique-des-resultats-de-la-recherche-financee-par-des-fonds-publics"
```

Le slug (raccourci) est produit de la manière suivante:
* on découpe les urls en morceaux a partir de "/" et on prend le dernier element de la liste de la manière suivante
```
liste_element_url = url.split("/")
slug = liste_element_url[-1]
```
    * vérifions pour la version originale:
```
liste_element_url = urlV_O.split("/")
print(liste_element _url)
slug = liste_element_url[-1]
print(slug)
```
On est censé avoir:
```
>>> ['http:', '', 'www.republique-numerique.fr', 'projects', 'projet-de-loi-numerique', 'consultation', 'consultation', 'opinions', 'section-2-travaux-de-recherche-et-de-statistique', 'article-9-acces-aux-travaux-de-la-recherche-financee-par-des-fonds-publics']
>>>'article-9-acces-aux-travaux-de-la-recherche-financee-par-des-fonds-publics'
```
    * vérifions pour une autre version:
```
liste_element_url = urlV_add.split("/")
print(liste_element _url)
slug = liste_element_url[-1]
print(slug)
```
On est censé avoir:
```
>>> ['http:', '', 'www.republique-numerique.fr', 'projects', 'projet-de-loi-numerique', 'consultation', 'consultation', 'opinions', 'section-2-travaux-de-recherche-et-de-statistique', 'article-9-acces-aux-travaux-de-la-recherche-financee-par-des-fonds-publics', 'versions', 'mise-a-disposition-systematique-des-resultats-de-la-recherche-financee-par-des-fonds-publics']
>>>'mise-a-disposition-systematique-des-resultats-de-la-recherche-financee-par-des-fonds-publics'
```

In [6]:
def load_versions(art_file="article9.json"):
    '''charger l'ensemble des versions de l'article'''
    versions_d = {}
    with open(art_file, "r") as f:
        article9 = json.load(f)
    
    #Pour rappel
    #print article9.keys()
    versions_d[article9["article_link"].split("/")[-1]] = {
                "id": 0,
                "date": article9["created_at"],
                "link": article9["article_link"],
                "slug": article9["article_link"].split("/")[-1],
                "title": article9["article_link"].split("/")[-1].replace("-", " "),
                "text": article9['body'],
                "author": article9["author"],
                "votes":[], 
                "arguments":article9["arguments"], 
                "votes_arguments": [],
                "sources":article9["sources"],
                "votes_sources": [],
                "total_votes": article9["votes_total"],
                #les décomptes de votes sur les arguments ne sont pas disponible dans l'article9
                  # seulement le nombre d'arguments
                "total_arguments_votes": article9["arguments_count"]
               }
    
    versions = article9["versions"]
    
    #on construit une liste de versions
    for i,v in enumerate(versions):
        #pour rappel
        #print v.keys()
        versions_d[v["slug"]] = {"date":v["created_at"],
                "id": i+1,
                "link":v["link"],
                #"slug":v["slug"],
                "title":v["title"],
                "text": v['comment'],
                "author": v["author"],
                "votes":[], 
                "arguments":[], 
                "votes_arguments": [],
                "sources":[],
                "votes_sources": [],
                "total_votes":v["votes_total"],
              #ici il s'agit bien du nombre de votes sur un argument
                "total_arguments_votes": v["arguments_count"]
             }
    print(len(versions_d)), "versions"
    return versions_d

# Les participants
Nous allons maintenant récupérer l'ensemble de la liste des participants au *projet de loi*

In [7]:
def load_participants():
    #charger les votes des participants
    with open("./participants.json", "r") as f:
        data = json.load(f)
        participants = data["participants"]
    
    participants_d = {}
    for part in participants:
        for k,v in part.items():
            
            participants_d[k] = v["votes"]
        
    print (len(participants_d), "participants sur l'ensemble des articles")
    return participants_d

### Les electeurs de l'article 9

Nous allons dans `update_version`
mettre à jour les version en ajoutant les votes de chaque particpants
Mais comme les votes sont organisés par electeurs
nous allons regarder chaque electeur:
    - verifier qu'il a voté pour une des versions de l'article 9
    identifiable par son nom générique 
```
art = "article-9-acces-aux-travaux-de-la-recherche-financee-par-des-fonds-publics"
``` 
    - vérifier pour quelle version il a voté:
    identifiable par le slug (cf. plus haut)
    - vérifier ensuite s'il s'agit d'un vote sur une source argument ou la version elle même
    identifiable par #source-<id> ou #arg-<id> ou s'il n'en a pas il s'agit de la version elle même
    

In [9]:
def update_versions(participants, versions_d, art="article-9-acces-aux-travaux-de-la-recherche-financee-par-des-fonds-publics"):
    #pour chaque participant
    for user, votes in participants.items():
        #pour chaque vote du participants
        for vote in votes:            
            link = vote["link"]
            #a-t-il voté pour notre article?
            if art in link:
                date =  vote["date"]
                opinion = vote["opinion"]
                #pour quelle version?
                slug = link.split("/")[-1]
                
                #est ce qu'on peut découper avec #?
                # = y a t'il une info complémentaire du type #arg #source
                try:
                    #oui: c'est un vote sur un arg ou une source
                    slug, tid = slug.split("#") 
                    #exemple:
                    
                    if "arg-" in tid:
                        #dans notre dictionnaire de référence sur les versions
                        #c'est un argument on stocke dans les votes pour les arguments
                        versions_d[slug]["votes_arguments"].append({"electeur":user,"vote": opinion, "date":date, "id":tid})
                    elif "source-" in tid:
                        #c'est une sources on stocke dans les votes pour les sources
                        versions_d[slug]["votes_sources"].append({"electeur":user,"vote": opinion, "date":date, "id":tid})
                    else:
                        #au cas ou il y aurait un #autrechose mais ne s'applique pas ici
                        print(slug)
                except:
                    #non: c'est donc un vote simple sur une version (originale ou autre)
                    versions_d[slug]["votes"].append({"electeur":user,"vote": opinion, "date":date})
    return versions_d
    

Ici il s'agit d'une simple fonction pour faciliter les statistiques 
et pour ne garder que ce qui nous interesse on verra ici que on ne s'intéresse qu'aux votes sur les versions

Petit rappel de repartition des votes par types:
    * votes sur les versions
    * votes sur les arguments de chaque versions
    * votes sur les sources de chaque versions

Rangés dans des listes qui contiennent la date, l'electeur, le vote et son id unique
mis à plat dans `load_votes()` dans des listes de votes.

In [13]:
def load_votes(participants,art ="article-9-acces-aux-travaux-de-la-recherche-financee-par-des-fonds-publics"):
    #charger les votes des electeurs de l'article 9
    votes_v = []
    votes_args = []
    votes_src = []
    for user, votes in participants.items():
        for vote in votes:
            link = vote["link"]
            if art in link:
                date =  vote["date"]
                opinion = vote["opinion"]
                slug = link.split("/")[-1]
                try:
                    slug, tid = slug.split("#") 
                    if "arg-" in tid:
                        votes_args.append({"electeur":user,"vote": opinion, "slug":slug, "date":date, "id":tid})
                    elif "source-" in tid:
                        votes_src.append({"electeur":user,"vote": opinion, "slug":slug, "date":date, "id":tid})
                    else:
                        print(slug)
                except:
                    votes_v.append({"electeur":user,"vote": opinion, "slug":slug, "date":date})
            
    print(len(votes_v),"votes", len(votes_args),"votes sur les arg", len(votes_src), "votes sur les sources")
    return votes_v, votes_args, votes_src
    

# Filter top_electeur
Premier objectif : calculer combien de fois un electeur a voté pour l'article (c'est à dire toutes les versions)
Deuxième objectif: qui sont les electeurs les plus actifs? 
Si on analyse la répartition des electeurs / X le nombre de votes, on se rend compte comme la plupart des datasets que 80% des electeurs ont voté pour moins de 20 % des votes
>> Dans l'ideal générer l'histogramme du nombre de vote par electeur a montrer pour justifier le seuil

Ici on filtre les electeurs les plus actifs: soit le SEUIL dit: 
l'electeur doit avoir voté au moins X fois

In [15]:
def filter_top_electeurs(votes, SEUIL):
    from collections import Counter, defaultdict
    #ici je crée la liste des electeurs uniques qui ont votés
    electeur_names = list(set([v["electeur"] for v in votes]))
    #ici je crée un dictionnaire pour chaque electeur qui contient une liste vide
    electeurs_d = defaultdict.fromkeys(electeur_names, [])
    
    #On déroule la liste des votes 
    for vote in votes:
        # chaque vote est un dictionnaire
        #qui contient le nom de l'electeur + ...
        # vote = {"electeur": "cdequatrebarbes", "opinion":1, "date": XXX, "slug": XXX, "id":5}
        #pour chaque electeur on lui ajoute son vote (ici un type de vote choisi (cf.votes))
        electeurs_d[vote["electeur"]].append(vote)
    
    #ici on compte simplement le nombre de nom dans la liste 
    f = Counter([data["electeur"] for data in votes])
    nb_votes_top_users = 0
    #f["vincentreverdy"] x fois dans la liste
    #ici ce qu'on veut c'est le 
    for name,nb_votes in f.items():
        if nb_votes < SEUIL:
            del electeurs_d[name]
            nb_votes_top_users +=nb_votes
    #Quelques stats pour justifier le seuil de participation
    nb_top_users =  len(electeurs_d)
    #on a aussi nb_votes_top_users
    nb_votes_total = len(votes)
    
    nb_total_users = len(electeur_names)
    
    
    print ("TOTAL ===============")
    print ("Nb Total de votes sur les versions", str(nb_votes_total))
    print ("Nb Total d'electeurs sur les versions", str(nb_total_users))
    print ("TOP USERS ===============")
    print ("Nb d'electeur ayant votés", SEUIL, "fois:", nb_top_users)
    print ("Nb de votes des electeurs ayant votés", SEUIL, "fois:", nb_votes_top_users)
    print ("POURCENTAGE ===============")
    print ("Ces electeurs ayant votés au moins", SEUIL, "fois\n représentent:")
    part_users = float(float(nb_top_users)/nb_total_users)*100
    part_votes = (float(nb_votes_top_users)/nb_votes_total)*100
    print ("- % des electeurs:", part_users)
    print ("- % des votes:",part_votes)
    return electeurs_d
    

In [16]:
## Calc_similarity
#on calcule la similarité entre les top_electeurs sur l'ensemble des versions
#on definit grace au SEUIL que les electeurs sont similaires
#a partir du moment où ils ont votés x fois sur la même version


In [17]:
def calc_similarity(electeurs, versions_d, SEUIL=0):
    #Rappel:definition du score de similarité
    #un electeur est similaire à un autre
    #dans la mesure où ils ont voté X fois de la même manière
    
    from itertools import combinations
    votes_by_version = {}
    #pour chaque version
    for v, k in versions_d.items():
        #j'ajoute les votes de chaque utilisateur:
        #versionA = [{"user x": 1}, {"user Y": -1}, {"user B": 0}]
        votes_by_version[v] = {e["electeur"]:e["vote"] for e in k["votes"]}
    
    #mon score de similarité est un dictionnaire de couple possible
    #avec un score à 0 au début
    similarity_score = {}
    #tous les couples uniques possibles 
    #(sans permutation c'est à dire sans distinction d'orientation)
    # soit pour une combinaison de couple(2)
    #["A","B", "C"]
    #combo = [(A, B), (A,C), (B,C)]
    #et non pas
    #permut = [(A, B),(B,A), (A,C),(C,A), (B,C), (C,B)]
    #notre graphe sera non dirigée le sens n'a aucune importance
    #vu qu'il s'agit de savoir s'ils sont identiques
    for couple in combinations(electeurs.keys(), 2):
        userA, userB = couple
        
        similarity_score[couple] = 0
        #pour chaque version
        for k,v in votes_by_version.items():
            #verifier si A et B ont tous les deux votés pour cette version
            if userA in v.keys() and userB in v.keys():
                #si A a voté comme B
                if v[userA] == v[userB]:
                    #alors on ajoute 1
                    similarity_score[couple] += 1
                
    #on recupère nos couples de potes si leur score de similarité
    # est superieur ou égal au SEUIL
    return {couple:score for couple,score in similarity_score.items() if score >= SEUIL}


In [18]:
def build_histogram(versions_nb):
    import pylab
    N = len(versions_nb)
    votes_infos = [v["total_votes"]  for v in versions_nb.values()]
    
    args_infos = [v["total_arguments_votes"]  for v in versions_nb.values()]
    plt.title("Historique des votes et votes d'arguments par version")
    plt.ylabel('Nb votes')
    #pylab.xlim([0,108])
    plt.plot(votes_infos,color='r')
    plt.plot(args_infos,color='b')

    plt.show()
    plt.title("historique des votes des arguments par version")
    plt.ylabel('Nb votes sur les arguments')
    pylab.xlim([0,len(versions_nb)])
    plt.plot(args_infos,color='b')
    plt.show()
    return

In [19]:
def build_distrib(freq):
    '''build repartition votes'''
    N = len(freq)
    plt.title("Nombre de votes par auteurs")
    plt.plot(sorted(freq.values()),color='r')
    plt.show()
    plt.title("historique des votes des arguments par version")
    plt.ylabel('Nb votes sur les arguments')
    pylab.xlim([0,N])
    pylab.ylim([1,50])
    plt.show()
    return 

In [20]:
def build_dataset(top_electeurs, vs, SEUIL=1):
    similar_couple = calc_similarity(top_electeurs, vs, SEUIL=SEUIL)
    with open("families.csv", "w") as f:
        f.write("userA\tuserB\tscore\n")
        for couple, score in similar_couple.items():
            userA, userB = couple
            f.write("\t".join([userA, userB, str(score)])+"\n")
    return similar_couple

In [22]:
def build_graph(simil, top_users, SEUIL, name="test.gexf"):
    #instancier mon graph non dirigé
    g = nx.Graph()
    #spatialisation selon l'algo Fruchterman Reingold
    pos = nx.spring_layout(g)
    #pour chaque user du couple
    for couple, score in simil.keys():
        #j'ajoute un noeud 
        #qui a pour poids le nombre de votes de l'utilisateur
        g.add_node(couple[0], nb_votes=str(len(top_users[couple[0]])))
        g.add_node(couple[1], nb_votes=str(len(top_users[couple[1]])))
        #j'ajoute le lien entre les noeuds
        #qui a pour poids le score de similarité
        g.add_edge(couple[0], couple[1], score=str(score))
    
    for node in nx.isolates(g):
        print ("Isolé", node)
        g.remove_node(node)
    #name = "graph_similarity_TOP%i_SCORE_%i.gexf" %(SEUIL)
    nx.write_gexf(g, name)
    nx.draw(g)
    plt.show()
    return

In [28]:
def build_graph_from_db(top_users, SEUIL, ):
    name = "./carto_%i_%i.gexf" %SEUIL
    import csv
    import networkx as nx
    g = nx.Graph()
    pos = nx.spring_layout(g)
    with open("families.csv", "r") as f:
        r = csv.reader(f, delimiter="\t")
        for i, row in enumerate(r):
            
            if i == 0:
                pass
            else:
                if int(row[2]) >= SEUIL[1]:
                    poid_userA = len(top_users[row[0]])
                    poid_userB = len(top_users[row[1]])
                    g.add_node(row[0], votes_nb=len(top_users[row[0]]))
                    g.add_node(row[1], votes_nb=len(top_users[row[1]]))
                    g.add_edge(row[0], row[1], score=row[2])
    
    print (name)
    nx.write_gexf(g, name)
    return name
                    

In [31]:
if __name__ == "__main__":
    #versions_d = build_versions()
    #votes, electeurs_d = load_electeurs()
    vs = load_versions("./article_9.json")
    part = load_participants()
    vs = update_versions(part, vs)
    votes, vvargs, vsources= load_votes(part)
    #build_histogram(vs)
    #build_distrib(vs)
    
    #SEUIL
    SEUIL = (10, 3)
    #seuil 0 : Qui on considère comme top_user (i.i combine de votes minimum?, 
    #seuil1: A partir de combien de votes on peut dire qu'ils ont un comportement similaire?
    #Top electeurs (ont votes x fois au minimum)
    top_electeurs = filter_top_electeurs(votes, SEUIL[0])
    #On stocke les couples similaires 
    #(les couples de top_electeur ont votés 1 fois pareil au minimum)
    #similar_couples = build_dataset(top_electeurs, vs)
    build_graph_from_db(top_electeurs, SEUIL)
    
                    
                    
            
        
    #filtered_sim_couples = {k: v for k,v in similar_couples.items() if v >= SEUIL[1]}
    #graph = build_graph(filtered_sim_couple, top_electeurs)

109
21329 participants sur l'ensemble des articles
12851 votes 2522 votes sur les arg 174 votes sur les sources
TOTAL ===============
Nb Total de votes sur les versions 12851
Nb Total d'electeurs sur les versions 5635
TOP USERS ===============
Nb d'electeur ayant votés 10 fois: 126
Nb de votes des electeurs ayant votés 10 fois: 10554
POURCENTAGE ===============
Ces electeurs ayant votés au moins 10 fois
 représentent:
- % des electeurs: 2.236024844720497
- % des votes: 82.1259045988639


KeyError: 'danielbourrion'